In [31]:
import requests
import json

headers = {"Authorization": "Bearer api_key"}
# url="https://api.edenai.run/v2/ocr/ocr_tables_async"

# file_url = "https://developer.mozilla.org/en-US/docs/Learn/HTML/Tables/Basics/numbers-table.png"
# provider = "amazon"

# payload={
#     "providers": provider,
#     "file_url": file_url,
#     "language":"en",
# }

# response = requests.post(url, json=payload, headers=headers)
# result = response.json()
file_path = r"D:\Gen AI IEEE\OCR Table\2.jpg"

url = "https://api.edenai.run/v2/ocr/ocr_tables_async"
data = {
    "providers": "microsoft,google",
    'language': 'en'
}
files = {'file': open(file_path, 'rb')}

response = requests.post(url, data=data, files=files, headers=headers)

result = json.loads(response.text)

print(result.keys())
job_id = result['public_id']

dict_keys(['public_id', 'status', 'error', 'results'])


In [32]:
import time

def poll_ocr_table_job(job_id: str, max_poll_count = 10, poll_interval_sec = 5) -> dict:
    """
    Poll asynchronous job every `poll_interval_sec` seconds
    Raises Exception if job still not finished after `max_poll_count`
    """
    for i in range(max_poll_count):
        time.sleep(poll_interval_sec)
        response = requests.get(f"{url}/{job_id}", headers=headers)
        data = response.json()
        if data['status'] == 'finished':
            return data
    raise Exception("Call took too long.")


poll_response = poll_ocr_table_job(job_id)
# we know there is only one page and one table
# in reality you can iterate over pages and create one cv file per table found
json_table = poll_response['results']["microsoft"]['pages'][0]['tables'][0]

In [33]:
csv_table = []
for row in json_table['rows']:
    csv_row = []

    for cell in row['cells']:
        csv_row.append(cell['text'])

    csv_table.append(csv_row)

In [34]:
import csv
import os

file_name = os.path.basename(file_path)

with open(f"{file_name}.csv", 'w') as csvfile:
    tablewriter = csv.writer(csvfile)
    tablewriter.writerows(csv_table)